In [ ]:
import boto3
import time

Credentials to access AWS (__aws_access_key_id__ and __aws_secret_access_key__) are stored in ~/.aws/credentials under [datalord] profile

In [ ]:
session = boto3.Session(profile_name='datalord', region_name='us-east-1')
client = session.client('redshift')

Update the variables in the cell below. Adjust for your needs.

In [ ]:
cluster_identifier = 'datalord-rs-cluster'
cluster_roles = [
    'arn:aws:iam::...AccountNumber...:role/DataLord-RH-role', # replace ...AccountNumber... with your real one
]
cluster_subnet_group_name = 'datalord-sub-gr'
cluster_subnet_group_descr = 'Datalord Redshift Subnet Group'
cluster_subnet_ids = [
    'subnet-...SubId...', # replace ...SubId... with your real one
]
user_name = 'datalord'
user_password = 'Password123' # just a sample. Change it!!!

Create Cluster Subnet Group<br>
Documentation for [create_cluster_subnet_group](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift.html#Redshift.Client.create_cluster_subnet_group)

In [ ]:
response = client.create_cluster_subnet_group(
    ClusterSubnetGroupName=cluster_subnet_group_name,
    Description=cluster_subnet_group_descr,
    SubnetIds=cluster_subnet_ids,
)
response['ClusterSubnetGroup']

Create Redshift cluster<br>
Documentation for [create_cluster](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift.html#Redshift.Client.create_cluster)

In [ ]:
response = client.create_cluster(
    ClusterIdentifier=cluster_identifier,
    ClusterType='single-node',
    NodeType='dc2.large',
    MasterUsername=user_name,
    MasterUserPassword=user_password,
    ClusterSubnetGroupName=cluster_subnet_group_name,
    PubliclyAccessible=True,
    IamRoles=cluster_roles,
)
response['Cluster']

Run the cell if you want to wait until the cluster is provision and available

In [ ]:
start_time = time.time()
cluster_available_waiter = client.get_waiter('cluster_available')
cluster_available_waiter.wait(ClusterIdentifier=cluster_identifier, 
                                            WaiterConfig={'Delay': 10, 'MaxAttempts': 999}
                             )
print(f'Processing time: {time.time() - start_time} s')